# Test notebook

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import folium
import streamlit as st
import os
import matplotlib
from rasterstats import zonal_stats
import rasterio
import requests

from src.utils import *
from src.utils_population import *
from src.utils_plotting import *

2023-04-03 18:22:19.281 
  command:

    streamlit run /home/daniele/miniconda3/envs/proximity/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]


## General parameters

In [3]:
verbose = True

## User input

In [4]:
# Parameters given by the user
# gdf_file = "test_data/mda_isochrones.geojson"
# gdf_file = 'test_data/mda_isochrones.zip'
gdf_file = 'test_data/per_admn_ad3_py_s1_ign_pp_district.zip'
# gdf_file = 'test_data/shakeintensity_complex.zip'
data_type = "UNadj_constrained"
year = 2020
aggregated = False

## Process input data

In [8]:
# Create geopandas dataframes and prepare starting points
gdf, error = load_gdf(gdf_file)

if error:
    raise ValueError(error)
    
# gdf = gdf.iloc[:5, ]

In [9]:
gdf

,Shape_Leng,Shape_Area,ADM3_ES,ADM3_PCODE,ADM3_REF,ADM3ALT1ES,ADM3ALT2ES,ADM2_ES,ADM2_PCODE,ADM1_ES,ADM1_PCODE,ADM0_EN,ADM0_ES,ADM0_PCODE,date,validOn,validTo,geometry
0,0.820572,0.023937,Abancay,PE030101,None,None,None,Abancay,PE0301,Apurimac,PE03,Peru,Perú (el),PE,2015-07-24,2020-07-14,None,"POLYGON ((-72.93158 -13.55616, -72.93140 -13.5..."
1,0.137948,0.000983,Abelardo Pardo Lezameta,PE020502,None,None,None,Bolognesi,PE0205,Ancash,PE02,Peru,Perú (el),PE,2015-07-24,2020-07-14,None,"POLYGON ((-77.14740 -10.29675, -77.14643 -10.2..."
2,1.687027,0.097424,Acari,PE040302,None,None,None,Caraveli,PE0403,Arequipa,PE04,Peru,Perú (el),PE,2015-07-24,2020-07-14,None,"POLYGON ((-74.61542 -15.13728, -74.61174 -15.1..."
3,0.726321,0.021049,Acas,PE021402,None,None,None,Ocros,PE0214,Ancash,PE02,Peru,Perú (el),PE,2015-07-24,2020-07-14,None,"POLYGON ((-77.26312 -10.43537, -77.26356 -10.4..."
4,0.824658,0.019976,Accha,PE081002,None,None,None,Paruro,PE0810,Cusco,PE08,Peru,Perú (el),PE,2015-07-24,2020-07-14,None,"POLYGON ((-71.80912 -13.92427, -71.80901 -13.9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1868,0.670345,0.009150,Zarumilla,PE240301,None,None,None,Zarumilla,PE2403,Tumbes,PE24,Peru,Perú (el),PE,2015-07-24,2020-07-14,None,"POLYGON ((-80.22546 -3.43294, -80.22887 -3.435..."
1869,1.253854,0.044287,Zepita,PE210407,None,None,None,Chucuito,PE2104,Puno,PE21,Peru,Perú (el),PE,2015-07-24,2020-07-14,None,"POLYGON ((-69.13646 -16.35465, -69.13648 -16.3..."
1870,1.220081,0.052707,Zorritos,PE240201,None,None,None,Contralmirante Villar,PE2402,Tumbes,PE24,Peru,Perú (el),PE,2015-07-24,2020-07-14,None,"POLYGON ((-80.55398 -3.73355, -80.55359 -3.733..."
1871,0.588094,0.010770,Zuñiga,PE150516,Zuniga,None,None,Cañete,PE1505,Lima,PE15,Peru,Perú (el),PE,2015-07-24,2020-07-14,None,"POLYGON ((-75.95941 -12.84897, -75.96013 -12.8..."


## Run computation

In [10]:
ee.Initialize()

In [15]:
%%time

gdf_with_pop = add_population_data(
    gdf=gdf,
    data_type=data_type,
    tif_folder='test_data/pop_data',
    data_folder='data',
    year=year,
    aggregated=aggregated,
    clobber=False,
    verbose=True,
    text_on_streamlit=False,
    progress_bar=False,
    force_from_wpAPI=False,
    force_from_GEE=False,
)

Finding intersections with countries...
Intersected countries: BOL, ECU, PER, BRA, COL, CHL.
Country: PER.
Aggregating raster...
Done!
CPU times: user 15min 27s, sys: 45.9 s, total: 16min 13s
Wall time: 16min 18s


In [16]:
gdf_with_pop

,Shape_Leng,Shape_Area,ADM3_ES,ADM3_PCODE,ADM3_REF,ADM3ALT1ES,ADM3ALT2ES,ADM2_ES,ADM2_PCODE,ADM1_ES,...,pop_m_35,pop_m_40,pop_m_45,pop_m_50,pop_m_55,pop_m_60,pop_m_65,pop_m_70,pop_m_75,pop_m_80
0,0.820572,0.023937,Abancay,PE030101,None,None,None,Abancay,PE0301,Apurimac,...,1535,1406,1225,1025,885,763,615,380,287,361
1,0.137948,0.000983,Abelardo Pardo Lezameta,PE020502,None,None,None,Bolognesi,PE0205,Ancash,...,66,63,46,46,39,35,31,25,18,18
2,1.687027,0.097424,Acari,PE040302,None,None,None,Caraveli,PE0403,Arequipa,...,217,195,156,128,107,77,60,45,33,36
3,0.726321,0.021049,Acas,PE021402,None,None,None,Ocros,PE0214,Ancash,...,43,42,48,44,43,38,34,24,19,19
4,0.824658,0.019976,Accha,PE081002,None,None,None,Paruro,PE0810,Cusco,...,91,112,94,76,75,60,53,41,28,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1868,0.670345,0.009150,Zarumilla,PE240301,None,None,None,Zarumilla,PE2403,Tumbes,...,929,895,737,600,479,310,214,149,96,107
1869,1.253854,0.044287,Zepita,PE210407,None,None,None,Chucuito,PE2104,Puno,...,615,557,491,465,419,339,263,225,148,207
1870,1.220081,0.052707,Zorritos,PE240201,None,None,None,Contralmirante Villar,PE2402,Tumbes,...,467,451,441,381,276,203,150,134,98,99
1871,0.588094,0.010770,Zuñiga,PE150516,Zuniga,None,None,Cañete,PE1505,Lima,...,59,55,49,44,37,29,24,18,13,14


## Plot

In [ ]:
aggregated=aggregated
joint=False
col_label = 'ADM3_ES'
legend_title = 'Admin level'
plot_title = 'Admin level'

fig = plot_pop_data(
    gdf=gdf_with_pop, 
    col_label=col_label,
    legend_title=legend_title, 
    plot_title=plot_title,
    aggregated=aggregated,
    joint=joint,
    cmap_name='viridis'
    )

fig.show()

## Save

In [13]:
gdf_with_pop.to_file(f"{gdf_file.split('.')[0]}_with_pop.shp.zip", encoding='utf-8', driver="ESRI Shapefile")

## Check saved file

In [5]:
gdf_check = gpd.read_file(f"{gdf_file.split('.')[0]}_with_pop.shp.zip")